In [2]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys #to be able to click/input
from selenium.webdriver.support.ui import WebDriverWait #wait excplicit
from selenium.webdriver.support import expected_conditions as EC #wait explicit
from selenium.webdriver.common.by import By #find_element
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, ElementClickInterceptedException
from tabulate import tabulate
import pandas as pd
from pandas.errors import InvalidIndexError
import numpy as np

In [3]:
driver = webdriver.Chrome()

# Can change the date here (currently: 14-09-2024.15-09-2024)
url = 'https://www.traveloka.com/en-id/hotel/search?spec=13-09-2024.14-09-2024.1.1.HOTEL_GEO.107442.Yogyakarta.2' 

driver.get(url)

In [4]:
# Note: This will only catch hotels that have all three information completed (name, price, stars), otherwise it would be skipped (as intended)

driver.execute_script("window.scrollTo(0, 0);")
hotel = []
for i in range(0,300):
    found = False
    while not found:
        try:
            card = driver.find_element(By.CSS_SELECTOR, f"div[data-index='{i}']")
            found = True
        except NoSuchElementException:
            driver.execute_script("window.scrollBy(0, 1000);")
            time.sleep(1) 
    try:
        name = card.find_element(By.CLASS_NAME, 'css-4rbku5.css-901oao.css-cens5h.r-a5wbuh.r-ubezar.r-b88u0q.r-rjixqe.r-fdjqy7').text
        price = card.find_element(By.CLASS_NAME, 'css-901oao.r-a5wbuh.r-b88u0q.r-135wba7.r-1ff274t').text
        stars = card.find_element(By.CSS_SELECTOR, "div[data-id='tvat-starRating']")
        star = stars.find_elements(By.TAG_NAME, 'img')
        listhotel = {
                'Nama Hotel': name,
                'Harga Sewa/Malam': price,
                'Bintang': len(star)
        }
        hotel.append(listhotel)
    except NoSuchElementException as ads:
        continue

    if i % 50 == 0:
        print(f"Progress: {i} hotel information retrieval attempt(s) done")

Progress: 0 hotel information retrieval attempt(s) done
Progress: 50 hotel information retrieval attempt(s) done
Progress: 200 hotel information retrieval attempt(s) done
Progress: 250 hotel information retrieval attempt(s) done


In [6]:
#del df
df = pd.DataFrame(hotel)
df

,Nama Hotel,Harga Sewa/Malam,Bintang
0,Hotel Neo Malioboro by ASTON,Rp 1.209.091,3
1,Royal Malioboro by ASTON,Rp 1.651.876,4
2,The Rich Jogja Hotel,Rp 483.471,4
3,SATORIA Hotel Yogyakarta,Rp 495.868,4
4,Arte Hotel Malioboro Yogyakarta,Rp 661.157,3
...,...,...,...
230,PYRENEES Malioboro Hotel,Rp 703.525,2
231,Rumah Palagan Residence,Rp 185.950,2
232,Tirta Sanita Hotel,Rp 132.231,1
233,RedDoorz Plus @ Pogung Raya,Rp 136.001,2


In [197]:
df.to_csv('Weekend Sept Week 1 Traveloka (Hotel Yogyakarta).csv', index=False)